# Suspicious Rundll32 Activity
Detects suspicious process related to rundll32 based on arguments

## Rule Content
```
- title: Suspicious Rundll32 Activity
  id: e593cf51-88db-4ee1-b920-37e89012a3c9
  description: Detects suspicious process related to rundll32 based on arguments
  status: experimental
  references:
  - http://www.hexacorn.com/blog/2017/05/01/running-programs-via-proxy-jumping-on-a-edr-bypass-trampoline/
  - https://twitter.com/Hexacorn/status/885258886428725250
  - https://gist.github.com/ryhanson/227229866af52e2d963cf941af135a52
  tags:
  - attack.defense_evasion
  - attack.execution
  - attack.t1085
  author: juju4
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      CommandLine:
      - '*\rundll32.exe* url.dll,*OpenURL *'
      - '*\rundll32.exe* url.dll,*OpenURLA *'
      - '*\rundll32.exe* url.dll,*FileProtocolHandler *'
      - '*\rundll32.exe* zipfldr.dll,*RouteTheCall *'
      - '*\rundll32.exe* Shell32.dll,*Control_RunDLL *'
      - '*\rundll32.exe javascript:*'
      - '* url.dll,*OpenURL *'
      - '* url.dll,*OpenURLA *'
      - '* url.dll,*FileProtocolHandler *'
      - '* zipfldr.dll,*RouteTheCall *'
      - '* Shell32.dll,*Control_RunDLL *'
      - '* javascript:*'
      - '*.RegisterXLL*'
    condition: selection
  falsepositives:
  - False positives depend on scripts and administrative tools used in the monitored
    environment
  level: medium

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='process_command_line.keyword:(*\\rundll32.exe*\ url.dll,*OpenURL\ * OR *\\rundll32.exe*\ url.dll,*OpenURLA\ * OR *\\rundll32.exe*\ url.dll,*FileProtocolHandler\ * OR *\\rundll32.exe*\ zipfldr.dll,*RouteTheCall\ * OR *\\rundll32.exe*\ Shell32.dll,*Control_RunDLL\ * OR *\\rundll32.exe\ javascript\:* OR *\ url.dll,*OpenURL\ * OR *\ url.dll,*OpenURLA\ * OR *\ url.dll,*FileProtocolHandler\ * OR *\ zipfldr.dll,*RouteTheCall\ * OR *\ Shell32.dll,*Control_RunDLL\ * OR *\ javascript\:* OR *.RegisterXLL*)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()